In [7]:
import numpy as np
import matplotlib as plt
import sympy as sy

In [11]:
e = np.e
i = np.sqrt(-1)

C:\Users\matia\AppData\Local\Temp\ipykernel_10736\3335872027.py:2: RuntimeWarning: invalid value encountered in sqrt
  i = np.sqrt(-1)


In [5]:
def gamma_fun(a):
    t = sy.Symbol("t")
    return sy.integrate(g_integral(t, a), (t, 0, float('inf')))

def g_integral(t, a):
    return (e**(-t))*(t**(a-1))

hypergeometric function of second kind

In [6]:
def U(a, b, z):
    t = sy.Symbol("t")
    r = sy.integrate(U_integral(t, a, b, z), (t, 0, float('inf')))
    r /= gamma_fun(a)
    return r

def U_integral(t, a, b, z):
    r = (e**(-z*t))
    r *= t**(a-1)
    r *= (t+1)**(b-a-1)
    return r

# One entry

def U_x(x, k, kp):
    a = 1 - (i*kp)/(2*k)
    b = 2
    z = 2*k*x*i

    r = e**(-k*x*i)
    r *= x
    r *= U(a, b, z)

    return r

def Up_x(x, k, kp):

    return 1

In [12]:
def B(A, k ,kp, a):
    num = i*2*k
    den =  Up_x(a, k, kp) * e**(-i*k*a)
    den += i*k * U_x(a, k, kp) * e**(-i*k*a)
    return (num/den)*A

def C(A, k ,kp, a):
    num = 2*(i*k * (e**(-i*k*a)) * e**(i*k*a))
    den =  Up_x(a, k, kp) * e**(-i*k*a)
    den += i*k * U_x(a, k, kp) * e**(-i*k*a)
    return (num/den)*A

In [ ]:
def wave_1(x, k, kp, A):
    r = A * e**(i*k*a)
    r += B(A,k,kp,x) * e**(-i*k*a)
    return r

def wave_2(x, k, kp, A):
    return C(A,k,kp,x) * U_x(x, k, kp)

Example for $^{224}T$


In [ ]:
A = 224
a = 7.22 * 10**(-15)